In [1]:
import tensorflow as tf
import argparse
import os.path
from cifar100_models import *
from cifar100_input import *
from utils import *

tf.logging.set_verbosity(tf.logging.INFO)

#### SET THIS BEFORE RUNNING
NAME = "cifar100/cifar100_scm_001"
try:
    os.makedirs(CHECKPOINT.format(NAME))
except FileExistsError:
    pass

tf.reset_default_graph()
train_files, test_files = maybe_download_and_extract()
train_data = Cifar100Record(train_files)
test_data = Cifar100Record(test_files)

model = Cifar100ShallowConvolutionalModel(train_data.image, train_data.label)

try:
    checkpoints = sorted(["{}/{}".format(ROOT.format(NAME),x[:-5]) for x in os.listdir(ROOT.format(NAME)) if x.endswith(".meta")], 
                     key=lambda x: int(x.split("-")[-1]))
except FileNotFoundError:
    checkpoints = []
model.saver.recover_last_checkpoints(checkpoints)
try: latest_checkpoint = model.saver.last_checkpoints[-2]
except: latest_checkpoint = None

In [2]:
model.train(NAME, restore_point=latest_checkpoint)

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_001/model.ckpt-200500
Saving to:  ../models/cifar100/cifar100_scm_001/model.ckpt_201000
Saving to:  ../models/cifar100/cifar100_scm_001/model.ckpt_201500
INFO:tensorflow:../models/cifar100/cifar100_scm_001/model.ckpt-201500 is not in all_model_checkpoint_paths. Manually adding it.
Saving to:  ../models/cifar100/cifar100_scm_001/model.ckpt_202000
INFO:tensorflow:../models/cifar100/cifar100_scm_001/model.ckpt-202000 is not in all_model_checkpoint_paths. Manually adding it.
Saving to:  ../models/cifar100/cifar100_scm_001/model.ckpt_202500
INFO:tensorflow:../models/cifar100/cifar100_scm_001/model.ckpt-202500 is not in all_model_checkpoint_paths. Manually adding it.
Saving to:  ../models/cifar100/cifar100_scm_001/model.ckpt_203000
INFO:tensorflow:../models/cifar100/cifar100_scm_001/model.ckpt-203000 is not in all_model_checkpoint_paths. Manually adding it.
Saving to:  ../models/cifar100/cifar100_scm_001/model.ckpt_203